In [152]:
import random
class Simulation:
    
    def createPlayer(self, num):
        p = Player()
        p.number=num
        print("Enter Player "+str(num)+"'s name:")
        p.name=input()
        return(p)
        
    def __init__(self):
        print("Game Started")
        self.shop = Shop()
        self.p1=self.createPlayer(1)
        self.p2=self.createPlayer(2)
        self.p1.setHandler(ActionHandler(self.shop, self.p1, self.p2))
        self.p2.setHandler(ActionHandler(self.shop, self.p2, self.p1))
        self.shop.reset()
        self.shop.setup([self.p1,self.p2])
        while True:
            self.takeTurn(self.p1)
            if self.shop.checkEnd():
                break
            self.takeTurn(self.p2)
            if self.shop.checkEnd():
                break  
        if self.p1.getVP() > self.p2.getVP():
            print(str(self.p1)+" wins!")
        else:
            print(str(self.p2)+" wins!")
        self.shop.reset()
        self.p1.reset()
        self.p2.reset()
            
    def takeTurn(self, player):
        print("It is "+str(player)+"'s turn:'")
        player.actionPhase()
        player.buyPhase(self.shop)
        player.cleanupPhase()
        

In [153]:
class Player:
    def __init__(self):
        self.reset()
    
    def __repr__(self):
        return "Player "+self.name
    
    def setHandler(self, ah):
        self.actionHandler = ah
        
    def draw(self, quantity=1):
        
        #if you don't have enough cards, put discard into deck
        cardsLeft = sum(self.deck.values())
        if cardsLeft < quantity: 
            if cardsLeft > 0:
                self.takeCard(cardsLeft)
            for card in self.discard:
                self.deck[card] = self.discard[card]
            if quantity-cardsLeft > 0:
                self.takeCard(quantity-cardsLeft)
            self.discard = {}
        else:      
            #draw what you need from your deck into your hand
            self.takeCard(quantity)
            
    def takeCard(self, quantity):
        for i in range(quantity):
            drawnCard = random.choice(list(self.deck))
            self.deck[drawnCard] -= 1
            if self.deck[drawnCard] == 0:
                del self.deck[drawnCard]
            if drawnCard in self.hand:
                self.hand[drawnCard] += 1
            else:
                self.hand[drawnCard] = 1
    
    def actionPhase(self):
        self.actions = 1
        while self.actions > 0:
            actionCards = []
            for cardName in self.hand:
                if Card(cardName).type=="Kingdom":
                    actionCards.append(cardName)
            if len(actionCards)>0:
                self.takeAction(actionCards)
                self.actions -= 1
            else:
                break
        self.actions = 0
    
    def takeAction(self, actionCards):
            print("\tYou have "+str(self.actions)+" actions left")
            print("\tEnter the index of your desired action:")
            print("\t"+str(actionCards))
            decision = int(input())
            if decision != -1:
                chosenAction = actionCards[decision]
                self.actionHandler.use(chosenAction)
                self.discardCard(chosenAction)
            
            
    def buyPhase(self, shop):
        self.buys += 1
        #find all money in hand
        for cardName in self.hand:
            card = Card(cardName)
            if card.type == "Treasure":
                self.treasure += card.value * self.hand[cardName]
        #buy things with treasure
        while self.buys > 0:
            self.treasure -= self.makePurchase(shop)
            self.buys -= 1
        self.treasure = 0
        self.buys = 0
            
    def makePurchase(self, shop):        
        #find all cards you can afford and are available
        buyable = []
        for cardName in shop.cards:
            if Card(cardName).cost <= self.treasure and shop.cards[cardName] > 0:
                buyable.append(cardName)
        print("\tYou have "+str(self.buys)+" buys remaining with "+str(self.treasure)+" to spend")
        print("\tEnter the index of the card you wish to buy:")
        print("\t"+str(buyable))
        decision = int(input())
        if decision != -1:
            bought = buyable[decision]
            shop.deal(bought, self, self.discard)
            return Card(bought).cost
        else:
            self.buys = 0
            return 0
        
    def cleanupPhase(self):
        self.discardCard()
        self.draw(5)
    
    def discardCard(self, card=None):
        if not card:
            for card in self.hand:
                if card in self.discard:
                    self.discard[card] += self.hand[card]
                else:
                    self.discard[card] = self.hand[card]
            self.hand = {}
        else:
            if card in self.discard:
                self.discard[card] += 1
            else:
                self.discard[card] = 1
            self.hand[card] -= 1
            if self.hand[card] == 0:
                del self.hand[card]
    
    def getVP(self):
        total = 0
        for card in self.hand:
            if Card(card).type == "VP":
                total += self.hand[card] * Card(card).value
        for card in self.deck:
            if Card(card).type == "VP":
                total += self.deck[card] * Card(card).value
        for card in self.discard:
            if Card(card).type == "VP":
                total += self.discard[card] * Card(card).value
        return total
    
    def reset(self):
        self.name=""
        self.number=0
        self.deck={}
        self.discard={}
        self.hand={}
        self.actions=0
        self.buys=0
        self.treasure=0
    
    

In [154]:
class ActionHandler:
    def __init__(self, shop, user, opponent):
        self.shop = shop
        self.user = user
        self.opp = opponent
        
    def use(self, card):
        if card == "Cellar":
            print("\tHow many cards will you discard?")
            inHand = self.user.hand.copy()
            del inHand["Cellar"]
            print(inHand)
            n = int(input())
            if n > 0:
                print("\t\tEnter the index of the card you wish to discard:")
                for i in range(n):
                    inHand = list(filter(lambda x: x != "Cellar", self.user.hand))
                    print(inHand)
                    self.user.discardCard(inHand[int(input())])
                self.user.draw(n)
            self.user.actions += 1
            
        if card == "Market":
            self.user.draw(1)
            self.user.actions += 1
            self.user.buys += 1
            self.user.treasure += 1
        
        if card == "Militia":
            if "Moat" not in self.opp.hand:
                print("\t"+str(self.opp)+", enter the index of the card you wish to discard:")
                for i in range(2):
                    hand = list(self.opp.hand)
                    print(hand)
                    self.opp.discardCard(hand[int(input())]) 
            self.user.treasure += 2
        
        if card == "Mine":
            treasure = ["Copper", "Silver", "Gold"]
            inHand = list(filter(lambda x: x in treasure, self.user.hand))
            if len(inHand) > 0:
                print("Which would you like to mine?")
                print(inHand)
                choice = int(input())
                if choice > -1 and inHand[choice] != "Gold":
                    if inHand[choice] == "Copper":
                        self.user.hand["Copper"] -= 1
                        if self.user.hand["Copper"] == 0:
                            del self.user.hand["Copper"]
                        if "Silver" in self.user.hand:
                            self.user.hand["Silver"] += 1
                        else: 
                            self.user.hand["Silver"] = 1
                    if inHand[choice] == "Silver":
                        self.user.hand["Silver"] -= 1
                        if self.user.hand["Silver"] == 0:
                            del self.user.hand["Silver"]
                        if "Gold" in self.user.hand:
                            self.user.hand["Gold"] += 1
                        else: 
                            self.user.hand["Gold"] = 1
        
        if card == "Moat":
            self.user.draw(2)
            
        if card == "Remodel":
            if len(self.user.hand) > 0:
                print("Which card would you like to trash?")
                inHand = list(self.user.hand)
                inHand.remove("Remodel")
                print(inHand)
                choice = int(input()) 
                spending = Card(inHand[choice]).cost + 2
                buyable = []
                for cardName in self.shop.cards:
                    if Card(cardName).cost <= spending and self.shop.cards[cardName] > 0:
                        buyable.append(cardName)
                print("Which card would you like to obtain?")
                print(buyable)
                self.shop.deal(buyable[int(input())], self.user, self.user.discard)
        
        if card == "Smithy":
            self.user.draw(3)
            
        if card == "Village":
            self.user.draw(1)
            self.user.actions += 2
        
        if card == "Woodcutter":
            self.user.buys += 1
            self.user.treasure += 2
        
        if card == "Workshop":
            buyable = []
            for cardName in self.shop.cards:
                if Card(cardName).cost <= 4 and self.shop.cards[cardName] > 0:
                    buyable.append(cardName)
            print("Which card would you like to obtain?")
            print(buyable)
            self.shop.deal(buyable[int(input())], self.user, self.user.discard)    
            

In [155]:
class Card:
    #all possible cards
    options = {
        "Cellar": ["Cellar",2,"Kingdom"],
        "Market": ["Market",5,"Kingdom"],
        "Militia": ["Militia",4,"Kingdom"],
        "Mine": ["Mine",5,"Kingdom"],
        "Moat": ["Moat",2,"Kingdom"],
        "Remodel": ["Remodel",4,"Kingdom"],
        "Smithy": ["Smithy",4,"Kingdom"],
        "Village": ["Village",3,"Kingdom"],
        "Woodcutter": ["Woodcutter",3,"Kingdom"],
        "Workshop": ["Workshop",3,"Kingdom"],
        "Copper": ["Copper",0,"Treasure",1],
        "Silver": ["Silver",3,"Treasure",2],
        "Gold": ["Gold",6,"Treasure",3],
        "Estate": ["Estate",2,"VP",1],
        "Dutchy": ["Dutchy",5,"VP",3],
        "Province": ["Province",8,"VP",6]
    }
    
    def __init__(self, name):
        self.name = name
        card = self.options[name]
        self.cost = card[1]
        self.type = card[2]
        if len(card) == 4:
            self.value = card[3]
    
    def __repr__(self):
        return self.name
    
    

In [156]:
class Shop:
    def reset(self):
        #initialize all cards
        self.cards = {
            "Cellar":10,
            "Market":10,
            "Militia":10,
            "Mine":10,
            "Moat":10,
            "Remodel":10,
            "Smithy":10,
            "Village":10,
            "Woodcutter":10,
            "Workshop":10,
            "Copper":60,
            "Silver":40,
            "Gold":30,
            "Estate":14,
            "Dutchy":8,
            "Province":8
        }
    
    def setup(self, players):
        for player in players:
            #deal out the starting decks to each player
            self.deal("Copper", player, player.deck, 7)
            self.deal("Estate", player, player.deck, 3)
            #each player draws 5
            player.draw(5)
    
    def deal(self, card, player, destination, quantity=1):
        #give the player a card from the shop
        self.cards[card] = self.cards[card]-quantity #CHECK IF WE RUN OUT? 
        if card in destination:
            destination[card] += quantity
        else:
            destination[card] = quantity
            
    def checkEnd(self):
        return len(list(filter(lambda x: self.cards[x]==0 , self.cards)))==3 or self.cards["Province"]==0          
        
        
        

In [157]:
x=Simulation()

Game Started
Enter Player 1's name:
Nathan
Enter Player 2's name:
2
It is Player Nathan's turn:'
	You have 1 buys remaining with 2 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Moat', 'Copper', 'Estate']
-1
It is Player 2's turn:'
	You have 1 buys remaining with 2 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Moat', 'Copper', 'Estate']
-1
It is Player Nathan's turn:'
	You have 1 buys remaining with 5 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Market', 'Militia', 'Mine', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Estate', 'Dutchy']
1
It is Player 2's turn:'
	You have 1 buys remaining with 5 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Market', 'Militia', 'Mine', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Estate', 'Dutchy']
-1
It is Player Nathan's turn:'
	You have 1 actions left
	Enter the index of your desire

1
	You have 1 buys remaining with 1 to spend
	Enter the index of the card you wish to buy:
	['Copper']
-1
It is Player 2's turn:'
	You have 1 buys remaining with 4 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Militia', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Estate']
-1
It is Player Nathan's turn:'
	You have 1 actions left
	Enter the index of your desired action:
	['Village', 'Mine']
0
	You have 2 actions left
	Enter the index of your desired action:
	['Village', 'Mine', 'Smithy']
0
	You have 3 actions left
	Enter the index of your desired action:
	['Mine', 'Smithy', 'Market']
2
	You have 3 actions left
	Enter the index of your desired action:
	['Mine', 'Smithy', 'Market']
2
	You have 3 actions left
	Enter the index of your desired action:
	['Mine', 'Smithy']
0
	You have 2 actions left
	Enter the index of your desired action:
	['Mine', 'Smithy']
1
	You have 1 actions left
	Enter the index of your desired action:
	['M

0
	You have 1 actions left
	Enter the index of your desired action:
	['Market', 'Mine']
0
	You have 1 actions left
	Enter the index of your desired action:
	['Market', 'Mine']
0
	You have 1 actions left
	Enter the index of your desired action:
	['Mine', 'Smithy']
0
Which would you like to mine?
['Silver', 'Copper']
0
	You have 7 buys remaining with 10 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Militia', 'Mine', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Gold', 'Estate', 'Dutchy', 'Province']
6
	You have 6 buys remaining with 7 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Militia', 'Mine', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Gold', 'Estate', 'Dutchy']
6
	You have 5 buys remaining with 4 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Militia', 'Moat', 'Remodel', 'Smithy', 'Village', 'Woodcutter', 'Workshop', 'Copper', 'Silver'

5
	You have 7 buys remaining with 6 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Militia', 'Mine', 'Moat', 'Remodel', 'Smithy', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Gold', 'Estate', 'Dutchy']
5
	You have 6 buys remaining with 2 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Moat', 'Copper', 'Estate']
-1
It is Player 2's turn:'
	You have 1 actions left
	Enter the index of your desired action:
	['Cellar']
-1
	You have 1 buys remaining with 4 to spend
	Enter the index of the card you wish to buy:
	['Cellar', 'Militia', 'Moat', 'Remodel', 'Smithy', 'Woodcutter', 'Workshop', 'Copper', 'Silver', 'Estate']
-1
It is Player Nathan's turn:'
	You have 1 actions left
	Enter the index of your desired action:
	['Market', 'Mine']
0
	You have 1 actions left
	Enter the index of your desired action:
	['Mine']
0
Which would you like to mine?
['Gold']
-1
	You have 2 buys remaining with 4 to spend
	Enter the index of the card you wish to buy:
	['Cellar'